In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import clip
from torch.nn import functional as F
import torch.nn as nn
from torchvision import transforms
from PIL import Image
train = False
classes = None
pictures= None

def load_data():
    data_list = []
    label_list = []
    texts = []
    images = []
    
    if train:
        text_directory = "/root/autodl-tmp/EEG/EEG_Image_decode/datasets/THINGS/images_set/training_images"  
    else:
        text_directory = "/root/autodl-tmp/EEG/EEG_Image_decode/datasets/THINGS/images_set/test_images"

    dirnames = [d for d in os.listdir(text_directory) if os.path.isdir(os.path.join(text_directory, d))]
    dirnames.sort()
    
    if classes is not None:
        dirnames = [dirnames[i] for i in classes]

    for dir in dirnames:

        try:
            idx = dir.index('_')
            description = dir[idx+1:]
        except ValueError:
            print(f"Skipped: {dir} due to no '_' found.")
            continue
            
        new_description = f"{description}"
        texts.append(new_description)

    if train:
        img_directory = "/root/autodl-tmp/EEG/EEG_Image_decode/datasets/THINGS/images_set/training_images"
    else:
        img_directory ="/root/autodl-tmp/EEG/EEG_Image_decode/datasets/THINGS/images_set/test_images"
    
    all_folders = [d for d in os.listdir(img_directory) if os.path.isdir(os.path.join(img_directory, d))]
    all_folders.sort()

    if classes is not None and pictures is not None:
        images = []
        for i in range(len(classes)):
            class_idx = classes[i]
            pic_idx = pictures[i]
            if class_idx < len(all_folders):
                folder = all_folders[class_idx]
                folder_path = os.path.join(img_directory, folder)
                all_images = [img for img in os.listdir(folder_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
                all_images.sort()
                if pic_idx < len(all_images):
                    images.append(os.path.join(folder_path, all_images[pic_idx]))
    elif classes is not None and pictures is None:
        images = []
        for i in range(len(classes)):
            class_idx = classes[i]
            if class_idx < len(all_folders):
                folder = all_folders[class_idx]
                folder_path = os.path.join(img_directory, folder)
                all_images = [img for img in os.listdir(folder_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
                all_images.sort()
                images.extend(os.path.join(folder_path, img) for img in all_images)
    elif classes is None:
        images = []
        for folder in all_folders:
            folder_path = os.path.join(img_directory, folder)
            all_images = [img for img in os.listdir(folder_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
            all_images.sort()  
            images.extend(os.path.join(folder_path, img) for img in all_images)
    else:

        print("Error")
    return texts, images
texts, images = load_data()

/root/miniconda3/envs/EEG/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from PIL import Image
import os
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import open_clip
from matplotlib.font_manager import FontProperties

import sys
from diffusion_prior import *
from custom_pipeline import *
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

torch.cuda.empty_cache()
diffusion_prior = DiffusionPriorUNet(cond_dim=1024, dropout=0.1)

# Assuming generator.generate returns a PIL Image
generator = Generator4Embeds(num_inference_steps=4, device=device)
pipe = Pipe(diffusion_prior, device=device)
pipe.diffusion_prior.load_state_dict(torch.load(f'/root/autodl-tmp/EEG/EEG_Image_decode/Generation/fintune_ckpts/reconstruction/sub-08/diffusion_prior.pt', map_location=device))
sub = 'sub-08'
directory = f"Generation/generated_imgs/{sub}"
emb_eeg_test = torch.load(f'/root/autodl-tmp/EEG/EEG_Image_decode/Generation/SCFD_eeg_features_{sub}_test.pt')
for k in range(200):
    eeg_embeds = emb_eeg_test[k:k+1]
    h = pipe.generate(c_embeds=eeg_embeds, num_inference_steps=50, guidance_scale=5.0)
    for j in range(10):
        image = generator.generate(h.to(dtype=torch.float16))
        # Construct the save path for each image
        path = f'{directory}/{texts[k]}/{j}.png'
        # Ensure the directory exists
        os.makedirs(os.path.dirname(path), exist_ok=True)
        # Save the PIL Image
        image.save(path)
        print(f'Image saved to {path}')
        torch.cuda.empty_cache()

Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  4.20it/s]
50it [00:00, 129.40it/s]
100%|██████████| 4/4 [00:00<00:00,  4.63it/s]


Image saved to Generation/generated_imgs/sub-08/aircraft_carrier/0.png


100%|██████████| 4/4 [00:00<00:00,  9.85it/s]


Image saved to Generation/generated_imgs/sub-08/aircraft_carrier/1.png


100%|██████████| 4/4 [00:00<00:00,  9.63it/s]


...

Image saved to Generation/generated_imgs/sub-08/wok/7.png


100%|██████████| 4/4 [00:00<00:00, 12.01it/s]


Image saved to Generation/generated_imgs/sub-08/wok/8.png


100%|██████████| 4/4 [00:00<00:00, 12.71it/s]


Image saved to Generation/generated_imgs/sub-08/wok/9.png


Load eeg and image embeddings